In [1]:
%matplotlib

Using matplotlib backend: <object object at 0x7fc70c039c30>


In [4]:
import pandas as pd
from tqdm.autonotebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import pickle

# plt.style.use(['science', 'notebook'])

## Preambula
We are going to run some computations, to save time let's create a decorator that stores and read from disk

In [5]:
def maybe_load_from_disk(location: Path):
    def decorator(func):
        def _inner(*args, **kwargs):
            if location.exists():
                print(f"[INFO] loading from {location}")
                with open(location, "rb") as f:
                    return pickle.load(f)
            res = func(*args, **kwargs)
            with open(location, "wb") as f:
                print(f"[INFO] saving to {location}")
                pickle.dump(res, f)
            return res
        return _inner
    return decorator
    a

In [6]:
import numpy as np
from pickle import load, dump
from pathlib import Path
from tqdm import tqdm
import torch
from torchvision.utils import make_grid
import torchvision.transforms as T
from torchvision.transforms.functional import to_pil_image

from PIL import Image
from multiprocessing import Pool

def get_data(root: Path):
    data = None
    for encode_path in tqdm(list(root.glob("*.pk"))):
        with open(encode_path, "rb") as f:
            batch = load(f)
            batch["indxs"] = batch["indxs"].numpy()
        if data is None:
            data = batch
        else:
            data["x"] = np.concatenate([data["x"], batch["x"]])
            data["indxs"] = np.concatenate([data["indxs"], batch["indxs"]])
            data["image_paths"] += batch["image_paths"]
        yield data
   

def pca(x, k, center=True):
    # center
    if center:
        m = x.mean(0, keepdim=True)
        s = x.std(0, unbiased=False, keepdim=True)
        x -= m
        x /= s
    # decomposition
    U, S, V = torch.linalg.svd(x)
    reduced = torch.mm(x, V[:k].T)

    return reduced


/home/zuppif/anaconda3/envs/dl/lib/python3.9/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [15]:
from torch.utils.data import TensorDataset, DataLoader
from tqdm.autonotebook import tqdm
x = None

for data in tqdm(get_data(Path("./encoded/"))):
    if x is None:
        x = data['x']
    else:
        x = np.concatenate([x, data["x"]])

x.shape



0it [00:00, ?it/s]

  0%|          | 0/335 [00:00<?, ?it/s]

(6815953, 512)

In [16]:
x = torch.from_numpy(x).float()

In [18]:
import random
idxs = random.sample(range(x.shape[0]), 32 * 100)
idxs = torch.tensor(idxs)
sampled_x = x[idxs]

sampled_x.shape

torch.Size([3200, 512])

In [125]:
torch.save(sampled_x, "sampled_x.pkl")

In [7]:
sampled_x = torch.load("sampled_x.pkl")

In [45]:
from torchmetrics.functional import pairwise_cosine_similarity
import seaborn
import matplotlib.pyplot as plt

sim_matrix = pairwise_cosine_similarity(sampled_x)

In [94]:
sim_matrix_norm = sim_matrix - sim_matrix.min(1, keepdim=True)[0]
sim_matrix_norm /= sim_matrix_norm.max(1, keepdim=True)[0]


In [107]:
reduced_x = pca(sampled_x, k=128)
print(f"x.shape={reduced_x.shape}")

x.shape=torch.Size([3200, 128])


In [108]:
reduced_sim_matrix = pairwise_cosine_similarity(reduced_x)

reduced_sim_matrix_norm = reduced_sim_matrix -  reduced_sim_matrix.min(1, keepdim=True)[0]
reduced_sim_matrix_norm /= reduced_sim_matrix_norm.max(1, keepdim=True)[0]


In [118]:
sim_sim_matrices = sim_matrix_norm @ reduced_sim_matrix_norm.t()

In [119]:
sim_sim_matrices.mean(), sim_sim_matrices.std(), sim_sim_matrices.min()

(tensor(0.7882), tensor(0.0758), tensor(0.4881))